In [1]:
import csv
import string
import re
from sklearn import svm
import sklearn
import gzip
import random
import itertools
from collections import defaultdict
import numpy as np
import datetime

#from sklearn.metrics import jaccard_similarity_score
#from scipy.sparse import csc_matrix

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.linear_model import LogisticRegression

import json

In [2]:
matches = []
count = 0
with open('dotaMatch.out', 'r') as f:
    for line in f:
        matches.append(json.loads(line))
# #     matches = [json.loads(line) for line in f]
#         count = count + 1
#         print(count)
        

In [23]:
# gets the hero that the player in the first slot played
len(matches)

132778

In [3]:
#['match_id', 'duration', 'radiant_win', 'players']
data = []
heroNames = {}
heroId = set()

for m in matches:
    if m['lobby_name'] == 'Ranked':
        datum = {}
        desiredKeys = ['match_id', 'duration', 'radiant_win']
        types = [int, int, lambda x: x == True]
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](m[desiredKeys[i]])

        datum['players'] = []
        for i in range(len(m['players'])):
            datum['players'].append(m['players'][i]['hero_id'])
            heroId.add(m['players'][i]['hero_id'])
            if 'hero_name' in m['players'][i]:
                heroNames[m['players'][i]['hero_name']] = m['players'][i]['hero_id']
        data.append(datum)
    


In [24]:
len(data)

67932

In [4]:
random.shuffle(data)
train = data[:60000]
val = data[60000:]

In [ ]:
train[0]['players']

In [ ]:

print(heroId)

In [ ]:
heroNames

In [19]:
print(matches[203]['match_id'])
print(matches[203]['players'][0]['hero_id'])
#Pangolier 120? and 119
#Dark Willow 114

heroNames['Pangolier'] = 115
heroNames['Dark Willow'] = 114

3594133657
119


In [20]:
idtohn = dict((v,k) for k,v in heroNames.items())

In [ ]:
idtohn

In [14]:
advantage = {}

smd = open('smd.csv')

for line in smd:
    shit = {}
    dudes = list(line.split(', '))
    dudes[2] = float(dudes[2][:4])
    
    if dudes[0] not in advantage:
        shit[dudes[1]] = dudes[2]
        advantage[dudes[0]] = shit
    else:
        advantage[dudes[0]][dudes[1]] = dudes[2]
    

In [ ]:
advantage

In [6]:
def accuracy(val_lbls, pred):
    numright = 0
    for i in range(len(val_lbls)):
        if val_lbls[i] == pred[i]:
            numright += 1
    return float(numright) / len(val_lbls)

def classify(tf, tl, vf, vl):
    classifiers = [
    #KNeighborsClassifier(3),
    svm.LinearSVC(C=1),
    #svm.LinearSVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=4),
    RandomForestClassifier(max_depth=4, n_estimators=50, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LogisticRegression()
    #QuadraticDiscriminantAnalysis()
    ]
    
    for clf in classifiers:
        clf.fit(tf, tl)
        val_predictions = clf.predict(vf)
        print (accuracy(vl, val_predictions))

In [7]:
maxid = 0

def baseline(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0]*230
        for p in d['players'][:5]:
            if p == 119 or p == 120:
                p = 115
#             if p > maxid:
            feat[p-1] = 1
        for p in d['players'][5:]:
            if p == 119 or p == 120:
                p = 115
#             if p > maxid:
            feat[p + 115-1] = 1
        return feat

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
        
    classify(train_feats, train_lbls, val_feats, val_lbls)
    



In [8]:
baseline(train, val)


0.5787947554210792
0.5358043368633384
0.5273575390821987
0.5724911749873929
0.5587493696419567
0.5689611699445285
0.578416540595058


In [21]:
def advbaseline(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0]*240
        
        #adv1 = 0
        #adv2 = 0
        
        for p in d['players'][:5]:
            if p == 119 or p == 120:
                p = 115
            feat[p-1] = 1
        for p in d['players'][5:]:
            if p == 119 or p == 120:
                p = 115
            feat[p + 115 -1] = 1
            
        ind = 230
        for p1 in d['players'][:5]:
            for p2 in d['players'][5:]:
                if p1 == 119 or p1 == 120:
                    p1 = 115
                    
                if p2 == 119 or p2 == 120:
                    p2 = 115
                    
                if p1 == p2:
                    continue
                    
                #print(idtohn[p1], idtohn[p2], d['match_id'])
                
                feat[ind] = feat[ind] + advantage[idtohn[p1]][idtohn[p2]]
            ind = ind + 1
            
                
        for p1 in d['players'][5:]:
            for p2 in d['players'][:5]:
                if p1 == 119 or p1 == 120:
                    p1 = 115
                    
                if p2 == 119 or p2 == 120:
                    p2 = 115
                    
                if p1 == p2:
                    continue
                
                #adv2 = adv2 + advantage[idtohn[p1]][idtohn[p2]]
                feat[ind] = feat[ind] + advantage[idtohn[p1]][idtohn[p2]]
            ind = ind + 1

        return feat

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    #print(train_feats)
        
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [22]:
# for evert character in team look at advantage(sum it) over other player, then sum that 
advbaseline(train, val)

0.5992183560262229
0.5347957639939486
0.5273575390821987
0.5939233484619264
0.5611447302067575
0.5694654563792234
0.597327281896117


In [ ]:
advantage['Dark Willow']

In [ ]:
advantage['Pangolier']